# End Objectives
- Refresh periodically or turn into an api?
- Scrape multiple sports(NBA, MMA, nrl, afl, SOCCER, TENNIS, DOGS and HORSES)
- Multiple Sports site comparisons (sportsbet, pointsbet, tab, ladbrokes)
- https://www.datacamp.com/community/tutorials/fuzzy-string-python string matching using fuzzy wuzzy
- Text or email you the notice (twillio)
- https://stackoverflow.com/questions/50777849/from-conda-create-requirements-txt-for-pip3 create a requirements text file

## Install Dependencies

In [148]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from selenium import webdriver
import time

## Gambling HTML

In [27]:
# Sportsbet URL NBA
sportsbet_url = 'https://www.sportsbet.com.au/betting/basketball-us/nba'
# TAB URL NBA
tab_url = 'https://www.tab.com.au/sports/betting/Basketball/competitions/NBA'
# Pointsbet url NBA
pointsbet_url = 'https://pointsbet.com.au/sports/basketball/NBA'
# Ladbrokes url NBA
ladbrokes_url = 'https://www.ladbrokes.com.au/sports/basketball/usa/nba'

In [162]:
# Loop through url and create a list of dictionaries from all the competition matches and then convert to df
# Change all the hard coded strings to variables when you find their attributes in other bookie sites
def get_sports_data(
    url, 
    bookie, 
    driver_path = 'C:/Users/sheng/Downloads/geckodriver-v0.30.0-win64/geckodriver',
    sleepy_time = 2):
    
    competitions = []    
    
    if bookie == 'sportsbet':
        html = requests.get(url).text
        soup = BeautifulSoup(html, 'html.parser')
        for comp in soup.find_all('div', 'card_fohmrj3'): # Loop through competitions container
            Odds = comp.find_all('div', re.compile('rice.ex')) # Find price text
            competitions.append(
            {
                'Bookie': bookie,
                'Time': comp.find('div', re.compile('time')).text,
                'Team_1': comp.select_one('div[data-automation-id="participant-one"]').text,
                'Team_2': comp.select_one('div[data-automation-id="participant-two"]').text,
                'Odds_1': Odds[0].text,
                'Odds_2': Odds[1].text
            })

    elif bookie == 'pointsbet': # JS enabled
        try:
            driver = webdriver.Firefox(executable_path = driver_path)
            driver.get(url)
            time.sleep(sleepy_time)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            for comp in soup.select('div.f1oyvxkl'):
                Odds = comp.select('span.fheif50')
                Teams = comp.select('div > div.fddsvlq')
                competitions.append(
                {
                    'Bookie': bookie,
                    'Time': comp.select_one('div span[class*="peds f1mz1"]').text[3:],
                    'Team_1': Teams[0].text,
                    'Team_2': Teams[1].text,
                    'Odds_1': Odds[0].text,
                    'Odds_2': Odds[3].text
                })
        finally:
            driver.quit()
            
    elif bookie == 'tab': # JS enabled
        try:
            driver = webdriver.Firefox(executable_path = driver_path)
            driver.get(url)
            time.sleep(sleepy_time)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            for comp in soup.find_all('div', 'template-item'):
                Odds = comp.find_all('div','animate-odd')
                # Exception handling for live events
                try:
                    Time = comp.find('li', attrs={'data-test': 'close-time'}).text
                    Odds_1 = Odds[1].text
                    Odds_2 = Odds[2].text
                except Exception:
                    Time = 'Live'
                    Odds_1 = 'na'
                    Odds_2 = 'na'
                Teams = comp.select_one('div span[class="match-name-text"]').text.split() # Teams come in string separated by 'v'
                index = Teams.index('v')
                competitions.append(
                {
                    'Bookie': bookie,
                    'Time': Time,
                    'Team_1': ' '.join(Teams[:index]),
                    'Team_2': ' '.join(Teams[index + 1:]),
                    'Odds_1': Odds_1,
                    'Odds_2': Odds_2
                })
        finally:
            driver.quit()

#     elif bookie == 'ladbrokes': # JS enabled
#         for comp in soup.find_all('div', 'template-item'):
#             Odds = comp.find_all('div','animate-odd').text
#             Teams = comp.select_one('div > span[class="match-name-text"]').split()
#             competitions.append(
#             {
#                 'Bookie': bookie,
#                 'Time': comp.find('li', re.compile('meta-data'), attrs={'data-test': 'close-time'}).text,
#                 'Team_1': Teams[0],
#                 'Team_2': Teams[2],
#                 'Odds_1': Odds[1].text,
#                 'Odds_2': Odds[2].text
#             })
            
    comp_df = pd.DataFrame(competitions)
    return comp_df

In [114]:
driver_path = 'C:/Users/sheng/Downloads/geckodriver-v0.30.0-win64/geckodriver'
driver = webdriver.Firefox(executable_path = driver_path)
driver.get(tab_url)
time.sleep(2)
html = driver.page_source
driver.quit()

In [155]:
soup = BeautifulSoup(html, 'html.parser')
competitions = []
for comp in soup.find_all('div', 'template-item'):
    Odds = comp.find_all('div','animate-odd')
    try:
        Time = comp.find('li', attrs={'data-test': 'close-time'}).text
    except Exception:
        Time = 'Live'
    Teams = comp.select_one('div span[class="match-name-text"]').text.split() # Teams come in string separated by 'v'
    index = Teams.index('v')
    competitions.append(
    {
        'Time': Time,
        'Team_1': ' '.join(Teams[:index]),
        'Team_2': ' '.join(Teams[index + 1:])
#         'Odds_1': Odds[1].text,
#         'Odds_2': Odds[2].text
    })


In [156]:
Odds

[<div class="animate-odd" ng-class="{
   'no-animation': oddsShouldChangeWithoutAnimation
 }" ng-repeat="odd in odds">1.95</div>,
 <div class="animate-odd" ng-class="{
   'no-animation': oddsShouldChangeWithoutAnimation
 }" ng-repeat="odd in odds">1.58</div>,
 <div class="animate-odd" ng-class="{
   'no-animation': oddsShouldChangeWithoutAnimation
 }" ng-repeat="odd in odds">2.45</div>,
 <div class="animate-odd" ng-class="{
   'no-animation': oddsShouldChangeWithoutAnimation
 }" ng-repeat="odd in odds">1.85</div>]

In [157]:
competitions

[{'Time': 'Live', 'Team_1': 'Golden State', 'Team_2': 'Boston'},
 {'Time': 'Live', 'Team_1': 'Sacramento', 'Team_2': 'Milwaukee'},
 {'Time': 'Live', 'Team_1': 'LA Clippers', 'Team_2': 'Toronto'},
 {'Time': 'Fri 18 Mar 10:10', 'Team_1': 'Orlando', 'Team_2': 'Detroit'}]

In [31]:
# TAB needs to be javascript enabled, use selenium to make a request, seems to be a git with selenium inscribed in scrapy class
from selenium import webdriver
import time

# Run firefox webdriver from executable path
driver = webdriver.Firefox(executable_path='C:/Users/sheng/Downloads/geckodriver-v0.30.0-win64/geckodriver')
# Get webpage
driver.get(pointsbet_url)
# Execute script to scroll to load and capture all JS elements
# driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
time.sleep(4)
html = driver.page_source
# driver.quit()
soup = BeautifulSoup(html, 'html.parser')
# with open('tab_html.txt', 'w') as f:
#     f.write(soup.prettify())
comp = soup.select('div.f1oyvxkl')

# comp = soup.find_all('div', attrs={'class':'template-item'})
comp

[<div class="f1oyvxkl" data-test="event"><a class="f1x4xdln f1i1gs4f fsw11lo" data-test="sportsPage0EventHeaderLink" href="/sports/basketball/NBA/1170690"><div class="fouabat"><div class="f1mxy1xb"><span class="f1ry9gvn"><svg class="fpz9ar8" height="18" stroke-width="1" viewbox="0 -64 1024 1024" width="18" xmlns="http://www.w3.org/2000/svg"><path d="M392.59 413.81c39.367-3.015 78.791-5.177 118.101-5.177 52.907 0 106.041 3.3 159.004 8.704-0.569 9.956-1.479 19.797-1.479 29.867 0 0.036 0 0.078 0 0.12 0 154.454 68.582 292.889 176.955 386.515l0.652 0.551c-89.035 78.019-206.433 125.611-334.948 125.611-0.045 0-0.090 0-0.135 0h0.007c-14.386-0.137-28.381-0.836-42.218-2.079l1.998 0.145c-7.68-27.477-77.141-285.070-77.938-544.256zM970.866 473.657c17.522 4.892 41.358 8.761 50.517 11.093-8.309 113.025-52.348 214.414-120.731 294.357l0.581-0.696c-94.569-79.798-154.226-198.39-154.226-330.915 0-0.103 0-0.205 0-0.308v0.016c0-6.713 0.683-13.312 0.967-19.968 86.476 12.346 161.85 28.558 235.263 49.429l-12.3

In [161]:
get_sports_data(pointsbet_url, bookie='pointsbet')

,Bookie,Time,Team_1,Team_2,Odds_1,Odds_2
0,pointsbet,"Tomorrow, 10:00am",Detroit Pistons,Orlando Magic,2.40,1.59


In [131]:
get_sports_data(sportsbet_url, bookie='sportsbet')

,Bookie,Time,Team_1,Team_2,Odds_1,Odds_2
0,sportsbet,LIVE,Milwaukee Bucks,Sacramento Kings,1.35,3.14
1,sportsbet,LIVE,Boston Celtics,Golden State Warriors,1.02,13.00
2,sportsbet,LIVE,Toronto Raptors,Los Angeles Clippers,1.17,4.90
3,sportsbet,"Friday, 18 Mar 10:10",Detroit Pistons,Orlando Magic,2.46,1.55


In [160]:
get_sports_data(tab_url, bookie='tab')

,Bookie,Time,Team_1,Team_2,Odds_1,Odds_2
0,tab,Fri 18 Mar 10:10,Orlando,Detroit,1.58,2.45


In [164]:
sportsB = get_sports_data(sportsbet_url, bookie='sportsbet')
tab = get_sports_data(tab_url, bookie='tab')
pointsB = get_sports_data(pointsbet_url, bookie='pointsbet')
sports_list = [sportsB, tab, pointsB]

In [165]:
pd.concat(sports_list)

,Bookie,Time,Team_1,Team_2,Odds_1,Odds_2
0,sportsbet,"Friday, 18 Mar 10:10",Detroit Pistons,Orlando Magic,2.34,1.61
0,tab,Fri 18 Mar 10:10,Orlando,Detroit,1.58,2.45
0,pointsbet,"Tomorrow, 10:00am",Detroit Pistons,Orlando Magic,2.40,1.59


In [ ]:
# Create a list of correct names for the sport (NBA, NRL) and wrangle data to match those
from fuzzywuzzy import fuzz


# Or somehow directly match odds across bookies

# Unique key for each set of matched odds

In [ ]:
# Test Function
def aggregator():
    list_2_agg = []
    for i in sports_list:
        list_2_agg.append(get_sports_data(i))

In [167]:
# data structure
data_url = {
    'Bookie': ['sportsbet', 'ladbrokes', 'pointsbet', 'tab'],
    'NBA': ['https://www.sportsbet.com.au/betting/basketball-us/nba','', '','https://www.tab.com.au/sports/betting/Basketball/competitions/NBA'],
    'MMA': ['','','','']
}
data_url_df = pd.DataFrame(data_url)

In [168]:
data_url_df

,Bookie,NBA,MMA
0,sportsbet,https://www.sportsbet.com.au/betting/basketbal...,
1,ladbrokes,,
2,pointsbet,,
3,tab,https://www.tab.com.au/sports/betting/Basketba...,


## Create a spider (crawler template) to crawl through different parts of sportsbet site

In [2]:
# Dependencies
import pandas as pd
import scrapy
from scrapy.spiders import CrawlSpider, Rule
from scrapy.crawler import CrawlerProcess
from scrapy.linkextractors import LinkExtractor

In [3]:
class BookieSpider(CrawlSpider):
    name = 'Bookie_Spider'
    
    # Boundaries of where our spider can crawl
    allowed_domains = ['sportsbet.com.au']
    
    # Starting URL for crawl
    start_urls = ['https://www.sportsbet.com.au/betting/sports-az']
    
    # Extract links from page based on tag elements
    sports_link_list = LinkExtractor(restrict_xpaths = ('//ul[contains(@class,"sectionsList_")]//div//a'),
                                    unique = True)
    
    # Establish set of rules
    rule_sports_list = Rule(sports_link_list,
                       follow = True)
    rule_sports_H2H_odds = Rule(callback = 'parse_item',
                                follow = False)
    
    # Create rules tuple
    rules = (
        rule_sports_list,
        rule_sports_H2H_odds
    )

    def parse_item(self, response):
#         competitions = []
        for comp in response.xpath('//div[contains(@class, "card_")]'):
            Odds = comp.css('div[class*="riceTex"] ::text').extract()
            competitions.append(
            {   # include sport, dont yield df, initiate dict outside of class
                'Time': comp.css('div[class*="time"]::text').extract(),
                'Team_1': comp.css('div[data-automation-id*="participant-one"] ::text').extract(),
                'Team_2': comp.css('div[data-automation-id*="participant-two"] ::text').extract(),
                'Odds_1': Odds[0],
                'Odds_2': Odds[1]              
            })
        comp_df = pd.DataFrame(competitions)

Create new rules and linkextractor functions for each of the following
* No tie Head 2 head odds
* Tie head 2 head odds
* List odds

In [3]:
class BookieSpider(CrawlSpider):
    name = 'Bookie_Spider'
    
    # Boundaries of where our spider can crawl
    allowed_domains = ['sportsbet.com.au']
    
    # Starting URL for crawl
    start_urls = ['https://www.sportsbet.com.au/betting/sports-az']
    
    # Different link extractor and rules for each sporting odds type - head to head or theres multiple or its +-
    # Extract links from page based on tag elements
    sports_link_list = LinkExtractor(restrict_xpaths = ('//ul[contains(@class,"sectionsList_")]//div//a'), # filter hrefs here or just fucking list the damn url
                                    unique = True)
    
    # Establish set of rules
    rule_sports_list = Rule(sports_link_list,
                       follow = False,
                       callback = 'parse_item') # False for now as it extracts data points from the first link in each section of the sports list, later adjusted to crawl through more links
#     rule_sports_H2H_odds = Rule(callback = 'parse_item',
#                                 follow = False)
    
    # Create rules tuple
    rules = (
        rule_sports_list,
#         rule_sports_H2H_odds
    )

    def parse_item(self, response):
        sport = response.css('head > title::text').extract()
        for comp in response.xpath('//div[contains(@class, "card_")]'):
            # Enter in the if conditions for different sports here?
            Odds = comp.css('div[class*="riceTex"] ::text').extract()

        yield {   # include sport, dont yield df, initiate dict outside of class
                'Sport': sport,
                'Time': comp.css('div[class*="time"] ::text').extract(),
                'Team_1': comp.css('div[data-automation-id*="participant-one"] ::text').extract(),
                'Team_2': comp.css('div[data-automation-id*="participant-two"] ::text').extract(),
                'Odds_1': Odds[0],
                'Odds_2': Odds[1]              
                }

In [7]:
competitions = []
process = CrawlerProcess()
process.crawl(BookieSpider)
process.start()

2022-03-11 23:06:23 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2022-03-11 23:06:23 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 21.7.0, Python 3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 21.0.0 (OpenSSL 1.1.1m  14 Dec 2021), cryptography 3.4.8, Platform Windows-10-10.0.22000-SP0
2022-03-11 23:06:23 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2022-03-11 23:06:23 [scrapy.crawler] INFO: Overridden settings:
{}
2022-03-11 23:06:23 [scrapy.extensions.telnet] INFO: Telnet Password: 9392dd9be8e326c1
2022-03-11 23:06:23 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2022-03-11 23:06:23 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 's

ReactorNotRestartable: 

In [25]:
import json

df = pd.read_json('C:/Users/sheng/OneDrive/Desktop/CodeDir/Financial_Portfolio_Tools/Data/test.jl', lines=True)
df

,Sport,Time,Team_1,Team_2,Odds_1,Odds_2
0,[NRL Betting Odds - NRL Ladder | Sportsbet],"[Sunday, 13 Mar 18:25]",[Nth Queensland Cowboys],[Canterbury Bulldogs],1.77,2.05
1,[Rugby Union Betting - Odds and Results | Spor...,"[07:05, | , French Top 14]",[],[],1.87,1.93
2,[Rugby League Betting - Odds and Results | Spo...,"[18:25, | , NRL]",[Nth Queensland Cowboys],[Canterbury Bulldogs],1.77,2.05
3,[Sports Novelties Betting - Odds and Results |...,"[11:00, | , AEW]",[],[],1.34,3.00
4,[UFC - MMA Betting - Odds and Results | Sports...,"[14:00, | , UFC Matches]",[],[],2.51,1.51
5,[Ice Hockey - US Betting - Odds and Results | ...,"[11:00, | , NHL Matches]",[Anaheim Ducks],[New Jersey Devils],1.98,1.83
6,[Australian Rules Betting - Odds and Results |...,"[19:40, | , AFL]",[West Coast],[Gold Coast],1.56,2.45
7,[ANZ Premiership betting odds - Netball | Spor...,"[Sunday, 13 Mar 14:10]",[Mainland Tactix],[Southern Steel],1.50,2.50
8,[Basketball - Aus/Other Betting - Odds and Res...,"[17:30, | , Australian NBL]",[Cairns Taipans],[Sydney Kings],3.04,1.35
9,[Falun betting odds - Cross Country | Sportsbet],"[Sunday, 13 Mar 22:15]",[],[],2.60,3.60


# Tests

In [ ]:
# CSS for sports elements with href urls "ul.sectionsList_f1ohqg01 div a"
# xpath for sports elements with href urls "//ul[contains(@class,"sectionsList_")]//div//a"

In [25]:
html = requests.get('https://www.sportsbet.com.au/betting/basketball-us/nba').text

2022-03-03 22:18:02 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.sportsbet.com.au:443
2022-03-03 22:18:02 [urllib3.connectionpool] DEBUG: https://www.sportsbet.com.au:443 "GET /betting/basketball-us/nba HTTP/1.1" 200 None


In [26]:
from scrapy import Selector
sel = Selector( text = html )

In [31]:
card = sel.xpath('//div[contains(@class, "card")]')

In [41]:
card.css('div[data-automation-id*="participant-two"] ::text').extract()

['Atlanta Hawks',
 'Brooklyn Nets',
 'Toronto Raptors',
 'Boston Celtics',
 'San Antonio Spurs',
 'Dallas Mavericks',
 'Los Angeles Clippers']

In [44]:
card.css('div[class*="riceTex"] ::text').extract()

['1.53',
 '2.51',
 '(-4.5)',
 '1.99',
 '(+4.5)',
 '1.82',
 'Over (+231.5)',
 '1.90',
 'Under (+231.5)',
 '1.90',
 '1.61',
 '2.33',
 '(-3.5)',
 '1.90',
 '(+3.5)',
 '1.90',
 'Over (+221.5)',
 '1.95',
 'Under (+221.5)',
 '1.86',
 '4.00',
 '1.25',
 '(+8.5)',
 '1.90',
 '(-8.5)',
 '1.90',
 'Over (+216.5)',
 '1.90',
 'Under (+216.5)',
 '1.90',
 '1.67',
 '2.22',
 '(-2.5)',
 '1.84',
 '(+2.5)',
 '1.97',
 'Over (+223.5)',
 '1.86',
 'Under (+223.5)',
 '1.95',
 '3.16',
 '1.37',
 '(+5.5)',
 '1.97',
 '(-5.5)',
 '1.84',
 'Over (+237.5)',
 '1.91',
 'Under (+237.5)',
 '1.89',
 '1.73',
 '2.10',
 '(-2.5)',
 '1.96',
 '(+2.5)',
 '1.85',
 'Over (+219.5)',
 '1.95',
 'Under (+219.5)',
 '1.86',
 '2.23',
 '1.66',
 '(+2.5)',
 '1.97',
 '(-2.5)',
 '1.84',
 'Over (+218.5)',
 '1.90',
 'Under (+218.5)',
 '1.90']

In [8]:
import requests
html = requests.get('https://www.sportsbet.com.au/betting/basketball-us/nba').text

from scrapy import Selector
sel = Selector( text = html )
print(sel.css('head > title::text').get())

NBA betting odds - Basketball - US | Sportsbet


## Arbitrage Opportunity Function

In [30]:
def arbitrage_opp(df):
    if (1/df.max()).sum()>0:
        print('There\'s arbitrage')

# Use Linear system matrix to solve for inequality for what values you can invest in for the odds 

## Expected Value Function

In [21]:
test = {'a':range(10),
       'b':range(10,20)}
test = pd.DataFrame(test)

In [28]:
1/test.max().sum()

0.03571428571428571

re.compile('nba')


## Twilio